In [1]:
from env import host, username, password
import os
import pandas as pd 
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from datetime import date

import nick_acquire as get 
import nick_wrangle as w

In [16]:
df = get.acquire_superstore_data()

Using cached CSV


In [21]:
#describe.describe_data(df)

In [23]:
df

,product_id,customer_id,order_id,order_date,ship_date,ship_mode,segment,city,state,postal_code,sales,quantity,discount,profit,customer_name,product_name,category,sub_category,region_name
0,FUR-BO-10001798,CG-12520,CA-2016-152156,2016-11-08,2016-11-11,Second Class,Consumer,Henderson,Kentucky,42420.0,261.9600,2.0,0.00,41.9136,Claire Gute,Bush Somerset Collection Bookcase,Furniture,Bookcases,South
1,FUR-CH-10000454,CG-12520,CA-2016-152156,2016-11-08,2016-11-11,Second Class,Consumer,Henderson,Kentucky,42420.0,731.9400,3.0,0.00,219.5820,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",Furniture,Chairs,South
2,FUR-TA-10000577,SO-20335,US-2015-108966,2015-10-11,2015-10-18,Standard Class,Consumer,Fort Lauderdale,Florida,33311.0,957.5775,5.0,0.45,-383.0310,Sean O'Donnell,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables,South
3,OFF-ST-10000760,SO-20335,US-2015-108966,2015-10-11,2015-10-18,Standard Class,Consumer,Fort Lauderdale,Florida,33311.0,22.3680,2.0,0.20,2.5164,Sean O'Donnell,Eldon Fold 'N Roll Cart System,Office Supplies,Storage,South
4,OFF-PA-10002365,AA-10480,CA-2017-114412,2017-04-15,2017-04-20,Standard Class,Consumer,Concord,North Carolina,28027.0,15.5520,3.0,0.20,5.4432,Andrew Allen,Xerox 1967,Office Supplies,Paper,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,OFF-BI-10002824,BN-11515,CA-2016-169943,2016-05-19,2016-05-24,Standard Class,Consumer,New York City,New York,10011.0,47.7440,4.0,0.20,14.9200,Bradley Nguyen,Recycled Easel Ring Binders,Office Supplies,Binders,East
1730,OFF-AP-10000828,JK-15370,CA-2015-127453,2015-12-19,2015-12-20,First Class,Consumer,Philadelphia,Pennsylvania,19143.0,434.3520,3.0,0.20,43.4352,Jay Kimmel,Avanti 4.4 Cu. Ft. Refrigerator,Office Supplies,Appliances,East
1731,OFF-AP-10002906,JK-15370,CA-2015-127453,2015-12-19,2015-12-20,First Class,Consumer,Philadelphia,Pennsylvania,19143.0,3.5520,2.0,0.20,0.4440,Jay Kimmel,Hoover Replacement Belt for Commercial Guardsman Heavy-Duty Upright Vacuum,Office Supplies,Appliances,East
1732,OFF-AP-10003860,JK-15370,CA-2015-127453,2015-12-19,2015-12-20,First Class,Consumer,Philadelphia,Pennsylvania,19143.0,88.8320,4.0,0.20,7.7728,Jay Kimmel,Fellowes Advanced 8 Outlet Surge Suppressor with Phone/Fax Protection,Office Supplies,Appliances,East


There are not any nulls or missing values to handle, which is good. I'll need to convert columns to snake_case.
The following columns are redundant: Region ID and Category ID because they're most descriptively captured via their respective names; Country is unnecessary, because everything is in the United States. 
Order_date and ship_date can be converted to date-time formats.
I can create month and year columns from the dates given. 

In [17]:
# Conversions: column names decapitalized, white-space converted to underscores
df.columns = [col.lower().replace(" ","_").replace("-","_") for col in df.columns]

In [18]:
df = df.drop(['region_id', 'category_id', 'country'], axis=1) 

In [25]:
df['order_date']= pd.to_datetime(df['order_date'])
df['ship_date']= pd.to_datetime(df['ship_date'])

In [26]:
df['days_bw_shipment'] = df['ship_date'] - df['order_date']

In [28]:
df = df.set_index('order_date').sort_index()

In [29]:
df['month'] = df.index.month_name()
df['year'] = df.index.year

In [30]:
df

,product_id,customer_id,order_id,ship_date,ship_mode,segment,city,state,postal_code,sales,...,discount,profit,customer_name,product_name,category,sub_category,region_name,days_bw_shipment,month,year
order_date,,,,,,,,,,,,,,,,,,,,,
2014-01-04,OFF-LA-10003223,PO-19195,CA-2014-112326,2014-01-08,Standard Class,Home Office,Naperville,Illinois,60540.0,11.784,...,0.2,4.2717,Phillina Ober,Avery 508,Office Supplies,Labels,Central,4 days,January,2014
2014-01-04,OFF-ST-10002743,PO-19195,CA-2014-112326,2014-01-08,Standard Class,Home Office,Naperville,Illinois,60540.0,272.736,...,0.2,-64.7748,Phillina Ober,SAFCO Boltless Steel Shelving,Office Supplies,Storage,Central,4 days,January,2014
2014-01-04,OFF-BI-10004094,PO-19195,CA-2014-112326,2014-01-08,Standard Class,Home Office,Naperville,Illinois,60540.0,3.540,...,0.8,-5.4870,Phillina Ober,GBC Standard Plastic Binding Systems Combs,Office Supplies,Binders,Central,4 days,January,2014
2014-01-09,TEC-AC-10001266,MS-17830,CA-2014-135405,2014-01-13,Standard Class,Consumer,Laredo,Texas,78041.0,31.200,...,0.2,9.7500,Melanie Seite,Memorex Micro Travel Drive 8 GB,Technology,Accessories,Central,4 days,January,2014
2014-01-09,OFF-AR-10004078,MS-17830,CA-2014-135405,2014-01-13,Standard Class,Consumer,Laredo,Texas,78041.0,9.344,...,0.2,1.1680,Melanie Seite,Newell 312,Office Supplies,Art,Central,4 days,January,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-30,OFF-AP-10002457,CC-12430,CA-2017-126221,2018-01-05,Standard Class,Home Office,Columbus,Indiana,47201.0,209.300,...,0.0,56.5110,Chuck Clark,"Eureka The Boss Plus 12-Amp Hard Box Upright Vacuum, Red",Office Supplies,Appliances,Central,6 days,December,2017
2017-12-30,OFF-BI-10003684,PO-18865,CA-2017-143259,2018-01-03,Standard Class,Consumer,New York City,New York,10009.0,52.776,...,0.2,19.7910,Patrick O'Donnell,Wilson Jones Legal Size Ring Binders,Office Supplies,Binders,East,4 days,December,2017
2017-12-30,TEC-PH-10004774,PO-18865,CA-2017-143259,2018-01-03,Standard Class,Consumer,New York City,New York,10009.0,90.930,...,0.0,2.7279,Patrick O'Donnell,Gear Head AU3700S Headset,Technology,Phones,East,4 days,December,2017


In [38]:
# Take it all and wrap it into a prepare function
def prepare_superstore(df):
    '''
    '''
    # First obtain the DataFrame and define the variable. 
    df = get.acquire_superstore_data()
    # Next convert column names to snake_case
    print('Data acquired: Initializing Preparatory Stage...')
    df.columns = [col.lower().replace(" ","_").replace("-","_") for col in df.columns]
    # drop redundant columns
    df = df.drop(['region_id', 'category_id', 'country'], axis=1)
    # Convert columns to date_time
    df['order_date']= pd.to_datetime(df['order_date'])
    df['ship_date']= pd.to_datetime(df['ship_date'])
    # Calculate days between shipment and order placement
    df['days_bw_shipment'] = df['ship_date'] - df['order_date']
    # Sort index by order date
    df = df.set_index('order_date').sort_index()
    # Create columns for month and year
    df['month'] = df.index.month_name()
    df['year'] = df.index.year
    print('Preparation complete. DF ready to be split for EDA.')
    # get the DF
    return df

In [39]:
df = get.acquire_superstore_data()

Using cached CSV


In [40]:
df = prepare_superstore(df)

Using cached CSV
Data acquired: Initializing Preparatory Stage...
Preparation complete. DF ready to be split for EDA.


In [41]:
df

,product_id,customer_id,order_id,ship_date,ship_mode,segment,city,state,postal_code,sales,...,discount,profit,customer_name,product_name,category,sub_category,region_name,days_bw_shipment,month,year
order_date,,,,,,,,,,,,,,,,,,,,,
2014-01-04,OFF-LA-10003223,PO-19195,CA-2014-112326,2014-01-08,Standard Class,Home Office,Naperville,Illinois,60540.0,11.784,...,0.2,4.2717,Phillina Ober,Avery 508,Office Supplies,Labels,Central,4 days,January,2014
2014-01-04,OFF-ST-10002743,PO-19195,CA-2014-112326,2014-01-08,Standard Class,Home Office,Naperville,Illinois,60540.0,272.736,...,0.2,-64.7748,Phillina Ober,SAFCO Boltless Steel Shelving,Office Supplies,Storage,Central,4 days,January,2014
2014-01-04,OFF-BI-10004094,PO-19195,CA-2014-112326,2014-01-08,Standard Class,Home Office,Naperville,Illinois,60540.0,3.540,...,0.8,-5.4870,Phillina Ober,GBC Standard Plastic Binding Systems Combs,Office Supplies,Binders,Central,4 days,January,2014
2014-01-09,TEC-AC-10001266,MS-17830,CA-2014-135405,2014-01-13,Standard Class,Consumer,Laredo,Texas,78041.0,31.200,...,0.2,9.7500,Melanie Seite,Memorex Micro Travel Drive 8 GB,Technology,Accessories,Central,4 days,January,2014
2014-01-09,OFF-AR-10004078,MS-17830,CA-2014-135405,2014-01-13,Standard Class,Consumer,Laredo,Texas,78041.0,9.344,...,0.2,1.1680,Melanie Seite,Newell 312,Office Supplies,Art,Central,4 days,January,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-30,OFF-AP-10002457,CC-12430,CA-2017-126221,2018-01-05,Standard Class,Home Office,Columbus,Indiana,47201.0,209.300,...,0.0,56.5110,Chuck Clark,"Eureka The Boss Plus 12-Amp Hard Box Upright Vacuum, Red",Office Supplies,Appliances,Central,6 days,December,2017
2017-12-30,OFF-BI-10003684,PO-18865,CA-2017-143259,2018-01-03,Standard Class,Consumer,New York City,New York,10009.0,52.776,...,0.2,19.7910,Patrick O'Donnell,Wilson Jones Legal Size Ring Binders,Office Supplies,Binders,East,4 days,December,2017
2017-12-30,TEC-PH-10004774,PO-18865,CA-2017-143259,2018-01-03,Standard Class,Consumer,New York City,New York,10009.0,90.930,...,0.0,2.7279,Patrick O'Donnell,Gear Head AU3700S Headset,Technology,Phones,East,4 days,December,2017


In [50]:
def split_superstore_data(df):
    '''
   Using methods gleaned from Codeup's Time Series Analysis lessons,
   splits the Superstore DF into Train Validate and Split; .5/.3/.2 respectively.
   Subsequently returns each.
    '''
    print('Dataframe Input received: Splitting Data .5/.3/.2.')
    train_size = int(len(df) * .5)
    validate_size = int(len(df) * .3)
    test_size = int(len(df) - train_size - validate_size)
    validate_end_index = train_size + validate_size

    # split into train, validation, test
    train = df[: train_size]
    validate = df[train_size : validate_end_index]
    test = df[validate_end_index : ]
    print(f'Train: {train.shape}, Validate {validate.shape}, and Test {test.shape} are ready.\
    \n Proceed with EDA.')
    return train, validate, test

In [51]:
train, validate, test = split_superstore_data(df)

Dataframe Input received: Splitting Data .5/.3/.2.
Train: (867, 21), Validate (520, 21), and Test (347, 21) are ready.    
 Proceed with EDA.


In [52]:
# Combine all into wrangle. 
import nick_wrangle as w

In [2]:
train, validate, test = w.wrangle_superstore()

NameError: name 'df' is not defined